In [1]:
import struct
import copy

import rospy
import gazebo_msgs.srv
import geometry_msgs.msg
import std_msgs.msg
import baxter_core_msgs.srv

from tf import transformations
import tensorflow as tf
import baxter_interface

import numpy as np
from utils import *
from baxter_pickplace_interface import *
from matplotlib import pyplot as plt
%matplotlib inline

/home/dof6/.local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [4]:
def quality_network_build(cropped_depth_image, gripper_quaternion, reuse=False, use_bn=True, network_type="large"):
    with tf.variable_scope("quality_network",reuse=reuse):
        if network_type is "large":
            h = tf.layers.conv2d(cropped_depth_image, 32, [7,7], activation=tf.nn.relu,
                                 kernel_initializer=tf.random_normal_initializer(stddev=0.01), name="conv1")
            if use_bn:
                h = tf.layers.batch_normalization(h)
            h = tf.layers.conv2d(h, 32, [7,7], activation=tf.nn.relu,
                                 kernel_initializer=tf.random_normal_initializer(stddev=0.01), name="conv2")
            if use_bn:
                h = tf.layers.batch_normalization(h)
            h = tf.layers.max_pooling2d(h, [2,2], [2,2], name="max1")
            h = tf.layers.conv2d(h, 32, [5,5], activation=tf.nn.relu,
                                 kernel_initializer=tf.random_normal_initializer(stddev=0.01), name="conv3")
            if use_bn:
                h = tf.layers.batch_normalization(h)
            h = tf.layers.conv2d(h, 32, [5,5], activation=tf.nn.relu,
                                 kernel_initializer=tf.random_normal_initializer(stddev=0.01), name="conv4")
            if use_bn:
                h = tf.layers.batch_normalization(h)
            h = tf.layers.max_pooling2d(h, [3,3], [2,2], name="max2")
            h = tf.layers.conv2d(h, 128, [5,5], activation=tf.nn.relu,
                                 kernel_initializer=tf.random_normal_initializer(stddev=0.01), name="conv5")
            if use_bn:
                h = tf.layers.batch_normalization(h)
            h = tf.reshape(h,[-1,128])
            h = tf.concat([h, gripper_quaternion], axis=1)
            h = tf.layers.dense(h, 256, activation=tf.nn.relu,
                                kernel_initializer=tf.random_normal_initializer(stddev=0.01), name="fc1")
            h = tf.layers.dense(h, 256, activation=tf.nn.relu,
                                kernel_initializer=tf.random_normal_initializer(stddev=0.01), name="fc2")
               
        quality_logit = tf.layers.dense(h, 1,
                                  kernel_initializer=tf.random_normal_initializer(stddev=0.01), name="quality_logit")
        quality = tf.sigmoid(quality_logit)
    return quality_logit, quality

cropped_depth_image_ph = tf.placeholder(tf.float32,[None,50,50,1])
gripper_quaternion_ph = tf.placeholder(tf.float32,[None,4])
_, quality_prediction = quality_network_build(cropped_depth_image_ph,gripper_quaternion_ph)

W0804 17:35:24.107898 140279524488960 deprecation.py:323] From <ipython-input-4-a338587a3bbd>:5: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0804 17:35:24.333322 140279524488960 deprecation.py:323] From <ipython-input-4-a338587a3bbd>:7: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
W0804 17:35:24.432956 140279524488960 deprecation.py:323] From <ipython-input-4-a338587a3bbd>:12: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0804 17:35:24